<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2006%20-%20Improving_Our_News_Articles_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q langchain==0.0.208 openai==0.27.8 python-dotenv newspaper3k lxml_html_clean==0.4.2

In [17]:
import os

# os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')

In [18]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()


try:
    response = session.get(article_url, headers=headers, timeout=10)

    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()

        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entirely new AI systems that can, for example, power real-time voice translations to large groups of people, each speaking a different language, so they can seamlessly collaborate on a research project or play an AR game together,” wrote Meta in a blog post.

“Ultimately, the work done with RSC will pave the way toward building technologies for the next major computing

In [19]:
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [20]:
from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [21]:
# generate summary
summary = chat(messages)
print(summary.content)

Summary:
- Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the world's fastest AI supercomputer.
- RSC is currently under construction and is expected to be fully operational by mid-2022.
- The supercomputer aims to train models with trillions of parameters, enhancing capabilities in natural language processing (NLP) and computer vision.
- Meta envisions RSC enabling real-time voice translations for collaborative projects and contributing to the development of the metaverse.
- RSC is projected to be 20 times faster than Meta's existing V100-based clusters and significantly improves training times for large-scale NLP workflows.
- A model with tens of billions of parameters can now train in three weeks, down from nine weeks.
- RSC incorporates security and privacy controls, allowing the use of real-world data for training, including identifying harmful content on Meta's platforms.
- Meta claims this is the first instance of addressing performance, reliability,

# ======

In [22]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List


# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary')
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [23]:
from langchain.prompts import PromptTemplate


# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# instantiate model class
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

chain = LLMChain(llm=model, prompt=prompt_template)

# Run the LLMChain to get the AI-generated answer
output = chain.run({"article_title": article_title, "article_text":article_text})

# Parse the output into the Pydantic model
parsed_output = parser.parse(output)
print(parsed_output)

title='Meta claims its new AI supercomputer will set records' summary=['Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the fastest AI supercomputer in the world.', 'RSC is currently under construction and is expected to be fully operational by mid-2022.', 'The supercomputer aims to train models with trillions of parameters, enhancing capabilities in natural language processing and computer vision.', 'Meta envisions RSC enabling real-time voice translations for collaborative projects and contributing to the development of the metaverse.', "RSC is projected to be 20 times faster than Meta's existing V100-based clusters and significantly improves training times for large-scale NLP workflows.", 'The infrastructure is designed with security and privacy in mind, allowing the use of real-world data for training.', 'Meta believes this is the first instance of addressing performance, reliability, security, and privacy at such a scale.']


In [25]:
parsed_output

ArticleSummary(title='Meta claims its new AI supercomputer will set records', summary=['Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the fastest AI supercomputer in the world.', 'RSC is currently under construction and is expected to be fully operational by mid-2022.', 'The supercomputer aims to train models with trillions of parameters, enhancing capabilities in natural language processing and computer vision.', 'Meta envisions RSC enabling real-time voice translations for collaborative projects and contributing to the development of the metaverse.', "RSC is projected to be 20 times faster than Meta's existing V100-based clusters and significantly improves training times for large-scale NLP workflows.", 'The infrastructure is designed with security and privacy in mind, allowing the use of real-world data for training.', 'Meta believes this is the first instance of addressing performance, reliability, security, and privacy at such a scale.'])